In [1]:
import json
import gzip
import nltk.data
import nltk.corpus
import nltk.stem
import re
import collections
import random
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook

In [2]:
with gzip.open('../data/simplewiki/simplewiki-20171103.parsed.norm.json.gz', 'rt', encoding='utf-8') as f:
    wiki = json.load(f)

In [4]:
def compute_degrees(wiki):
    # sort pages by ID
    pages = sorted(wiki.values(), key = lambda page: page['id'])
    
    # compute in-degrees
    counter = collections.Counter()
    for _, page in tqdm_notebook(wiki.items(), leave = False):
        for link in page['links']:
            target_page_id = wiki[link['target']]['id']
            counter[target_page_id] += 1
    in_degrees = [counter.get(i) or 0 for i in range(len(pages))]
    
    # compute out-degrees
    out_degrees = [len(page['links']) for page in pages]
    
    return in_degrees, out_degrees

In [5]:
in_degrees, out_degrees = compute_degrees(wiki)
non_empty_pages = (page for page in wiki.values() if len(page['text']) > 0)
top_3k_pages = sorted(non_empty_pages, key = lambda page: in_degrees[page['id']], reverse = True)[:3000]

HBox(children=(IntProgress(value=0, max=125947), HTML(value='')))

In [6]:
stopwords = set(nltk.corpus.stopwords.words('english'))
stemmer = nltk.stem.SnowballStemmer('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def word_tokenize(text):
    for word in nltk.word_tokenize(text):
        # skip stopwords
        if word in stopwords:
            continue
        
        # apply lemmatization
        word = lemmatizer.lemmatize(word)

        # apply stemming
        word = stemmer.stem(word)
        
        # skip non-words
        if not re.match(r'[a-z]', word):
            continue
        
        yield word

In [7]:
page_tfs = []
for page in tqdm_notebook(top_3k_pages, leave = False):
    page_tfs.append((page['id'], collections.Counter(word_tokenize(page['text']))))

HBox(children=(IntProgress(value=0, max=3000), HTML(value='')))

In [8]:
wiki_tfs = collections.Counter()
for _, counter in page_tfs:
    wiki_tfs.update(counter)

In [9]:
id_to_word_10k = [word for word, _ in wiki_tfs.most_common(10000)]
word_to_id_10k = { word: word_id for word_id, word in enumerate(id_to_word_10k) }

In [22]:
def compute_word_idfs(page_tfs, words):
    word_idfs = []
    for word in tqdm_notebook(words, leave = False):
        n = sum(1 for _, counter in page_tfs if counter[word] > 0)
        word_idfs.append(-np.log(n / len(page_tfs)))
    return word_idfs

In [30]:
word_idfs = compute_word_idfs(page_tfs, id_to_word_10k)

HBox(children=(IntProgress(value=0, max=10000), HTML(value='')))

In [33]:
def build_factorization_matrix(page_tfs, word_idfs, word_to_id_index):
    result = np.zeros([len(page_tfs), len(word_to_id_index)])
    
    # build TF matrix
    for i, (page_id, counter) in enumerate(page_tfs):
        for word, word_freq in counter.items():
            word_id = word_to_id_index.get(word)
            if word_id is None:
                continue
            result[i, word_id] = word_freq

    # normalize TFs
    norms = np.sum(result, axis = -1, keepdims = True)
    norms = np.maximum(1, norms)
    result /= norms
    
    # multiply by IDFs
    result *= np.reshape(word_idfs, [1, -1])
    
    return result

In [35]:
factorization_matrix = build_factorization_matrix(page_tfs, word_idfs, word_to_id_10k)

In [40]:
U, s, V = np.linalg.svd(factorization_matrix, full_matrices = False)
U.shape, s.shape, V.shape

((3000, 3000), (3000,), (3000, 10000))

In [41]:
def compute_approximation(U, s, V, k):
    s_k = np.array(s)
    s_k[k:] = 0
    return np.matmul(U, np.matmul(np.diag(s_k), V))

In [48]:
def compute_error_rates(actual, approx):
    errors = np.sum(
        np.abs(actual - approx),
        axis = -1,
        keepdims = True) / 2
    norms = np.sum(actual, axis = -1, keepdims = True)
    norms = np.maximum(norms, 1e-9)
    return errors / norms

In [49]:
def test_approximation(k):
    approx = compute_approximation(U, s, V, k)
    error_rates = compute_error_rates(factorization_matrix, approx)
    return np.percentile(error_rates, [1, 10, 25, 50, 75, 90, 99])

In [50]:
def test_approximations(ks):
    return np.stack([test_approximation(k) for k in ks])

In [51]:
test_approximations([1, 10, 100, 1000, 2000, 3000])

array([[  5.00350827e-01,   5.00540571e-01,   5.00715422e-01,
          5.01146240e-01,   5.02129284e-01,   5.03861365e-01,
          5.37439149e-01],
       [  4.69824156e-01,   7.77216879e-01,   8.40672246e-01,
          9.12692350e-01,   1.00879315e+00,   1.13196912e+00,
          1.58110920e+00],
       [  4.03113538e-01,   8.26816431e-01,   9.36405370e-01,
          1.09686814e+00,   1.31689470e+00,   1.63268948e+00,
          2.84907311e+00],
       [  3.46705829e-01,   9.65896027e-01,   1.24200840e+00,
          1.59300349e+00,   1.97867265e+00,   2.34254683e+00,
          2.98240312e+00],
       [  1.27916086e-01,   3.37307943e-01,   5.86064774e-01,
          1.04014358e+00,   1.35869301e+00,   1.60631104e+00,
          2.08562450e+00],
       [  2.27413610e-14,   2.73837692e-14,   3.08269521e-14,
          3.49972412e-14,   3.95543249e-14,   4.45635477e-14,
          5.83279850e-14]])